In [94]:
from fastbook import urlread, URLError, HTTPError, json, L, time
def search_images_ddg(term, max_images=200):
    "Search for `term` with DuckDuckGo and return a unique urls of about `max_images` images"
    assert max_images<1000
    url = 'https://duckduckgo.com/'
    res = urlread(url,data={'q':term})
    searchObj = re.search(r'vqd=([\d-]+)\&', res)
    assert searchObj
    requestUrl = url + 'i.js'
    params = dict(l='us-en', o='json', q=term, vqd=searchObj.group(1), f=',,,', p='1', v7exp='a')
    urls,data = set(),{'next':1}
    headers = dict(referer='https://duckduckgo.com/')
    while len(urls)<max_images and 'next' in data:
        try:
            res = urlread(requestUrl, data=params, headers=headers)
            data = json.loads(res) if res else {}
            urls.update(L(data['results']).itemgot('image'))
            requestUrl = url + data['next']
            # print(f'data: {requestUrl}\n')
        except (URLError, HTTPError, KeyError, ValueError): break# pass
        time.sleep(1)
    return L(urls)[:max_images]
    
def search_images(term, max_images=20):
    return search_images_ddg(term, max_images);

In [ ]:
urls = search_images('store receipt photos', 10)
urls[1]

In [84]:
from fastai.vision.all import Path
receipts_path = Path('./receipts')
Path.BASE_PATH = receipts_path
receipts_path.ls()

(#7) [Path('labels'),Path('receipt.jpg'),Path('images_done'),Path('receipts.tar.gz'),Path('.ipynb_checkpoints'),Path('via.json'),Path('images')]

In [73]:
%%script echo skipping
from fastdownload import download_url
dest = (receipts_path / 'receipt.jpg')

skipping


In [ ]:
%%script echo skipping
download_url(urls[1], dest, show_progress=False)

In [85]:
path = Path.BASE_PATH
dest = (path / 'images')
dest.mkdir(exist_ok=True, parents=True)

In [76]:
from fastai.vision.all import download_images
download_images(dest, urls=search_images(f'store receipt photo', 299))

data: https://duckduckgo.com/i.js?q=store%20receipt%20photo&o=json&p=1&s=100&u=bing&f=,,,&l=us-en

data: https://duckduckgo.com/i.js?q=store%20receipt%20photo&o=json&p=1&s=200&u=bing&f=,,,&l=us-en

data: https://duckduckgo.com/i.js?q=store%20receipt%20photo&o=json&p=1&s=300&u=bing&f=,,,&l=us-en

data: https://duckduckgo.com/i.js?q=store%20receipt%20photo&o=json&p=1&s=400&u=bing&f=,,,&l=us-en



In [101]:
from fastai.vision.all import verify_images, get_image_files

failed = verify_images(get_image_files(dest))
failed.map(Path.unlink)
len(failed)

0

In [97]:
import os
from PIL import Image
import imagehash
from datetime import datetime

def find_duplicate_images(dir_path, remove_duplicates=False, delete_only_today=True, verbose=True):
    # Dictionary to store hash values and their corresponding file paths
    hash_dict = {}
    today = datetime.now().date()
    
    def is_file_today(file_path):
        # Get the file's modification date
        mod_time = datetime.fromtimestamp(os.path.getmtime(file_path)).date()
        return mod_time == today

    def get_file_mod_time(file_path):
        # Get the file's modification timestamp
        return os.path.getmtime(file_path)
    
    # Walk through the directory
    for root, _, files in os.walk(dir_path):
        for file in files:
            if file.lower().endswith(('png', 'jpg', 'jpeg', 'gif', 'bmp', 'tiff')):
                file_path = os.path.join(root, file)
                
                # Open the image file and compute its hash
                try:
                    with Image.open(file_path) as img:
                        img_hash = imagehash.phash(img)
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")
                    continue
                
                # Check if the hash value already exists in the dictionary
                if img_hash in hash_dict:
                    existing_file_path = hash_dict[img_hash]
                    if verbose: print(f"Duplicate found: {file_path} and {existing_file_path}")
                    
                    if remove_duplicates:
                        file_path_mod_time = get_file_mod_time(file_path)
                        existing_file_mod_time = get_file_mod_time(existing_file_path)
                        
                        if delete_only_today:
                            if is_file_today(file_path) and is_file_today(existing_file_path):
                                if file_path_mod_time > existing_file_mod_time:
                                    file_to_remove = file_path
                                    file_to_keep = existing_file_path
                                else:
                                    file_to_remove = existing_file_path
                                    file_to_keep = file_path
                            elif is_file_today(file_path):
                                file_to_remove = file_path
                                file_to_keep = existing_file_path
                            elif is_file_today(existing_file_path):
                                file_to_remove = existing_file_path
                                file_to_keep = file_path
                            else:
                                continue
                        else:
                            if file_path_mod_time > existing_file_mod_time:
                                file_to_remove = file_path
                                file_to_keep = existing_file_path
                            else:
                                file_to_remove = existing_file_path
                                file_to_keep = file_path
                        
                        try:
                            os.remove(file_to_remove)
                            if verbose: print(f"Removed: {file_to_remove}")
                            hash_dict[img_hash] = file_to_keep  # Update the dictionary with the kept file path
                        except Exception as e:
                            print(f"Error removing {file_to_remove}: {e}")
                else:
                    hash_dict[img_hash] = file_path

In [102]:
%%capture
dir_path = dest
find_duplicate_images(dir_path, remove_duplicates=True, delete_only_today=True, verbose=False)

In [88]:
import os
import re

def find_highest_number(dir_path, prefix):
    highest_number = -1
    # Compile a regex pattern using the given prefix
    pattern = re.compile(rf'{re.escape(prefix)}(\d+)\.(png|jpg|jpeg|gif|bmp|tiff|PNG|JPG|JPEG|GIF|BMP|TIFF)$')

    for root, _, files in os.walk(dir_path):
        for file in files:
            match = pattern.match(file)
            if match:
                number = int(match.group(1))
                if number > highest_number:
                    highest_number = number

    return highest_number

In [103]:
dir_path = dest
prefix = 'receipt_'
highest_index = find_highest_number(dir_path, prefix)
print(f"The highest number with prefix '{prefix}' is: {highest_index}")

The highest number with prefix 'receipt_' is: 723


In [89]:
def rename_new_images(it, dest, prefix):
    image_files = get_image_files(dest)
    
    # Loop through each file and rename it
    for x in image_files:
        if not x.name.startswith(prefix):
            new_name = x.parent / (prefix + str(it) + x.suffix)
            x.rename(new_name)
            it += 1

In [104]:
rename_new_images(highest_index + 1, dest, prefix)

In [98]:
from fastai.vision.all import (download_images,
                                verify_images, 
                                get_image_files,
                                Path)
# search_images_ddg redefined previously
# find highest number
# rename_new_images

def auto_scrape(dest, search_terms, result_prefix, search_img_count=299):
    download_images(dest, urls=search_images(f'{search_terms}', search_img_count))
    
    failed = verify_images(get_image_files(dest))
    failed.map(Path.unlink)
    print(f'invalid file count: {len(failed)}')

    find_duplicate_images(dest, remove_duplicates=True, delete_only_today=True, verbose=False)

    highest_index = find_highest_number(dest, result_prefix)
    print(f"The highest number with prefix '{result_prefix}' is: {highest_index}")
    rename_new_images(highest_index + 1, dest, result_prefix)

In [99]:
auto_scrape(dest, 'store receipt photos', 'receipt_')

invalid file count: 41
The highest number with prefix 'receipt_' is: 663


In [100]:
while True: auto_scrape(dest, 'store receipt photos', 'receipt_')

invalid file count: 44
The highest number with prefix 'receipt_' is: 677
invalid file count: 48
The highest number with prefix 'receipt_' is: 688
invalid file count: 46
The highest number with prefix 'receipt_' is: 699
invalid file count: 38
The highest number with prefix 'receipt_' is: 704
invalid file count: 37
The highest number with prefix 'receipt_' is: 710
invalid file count: 36
The highest number with prefix 'receipt_' is: 712
invalid file count: 35
The highest number with prefix 'receipt_' is: 717
invalid file count: 39
The highest number with prefix 'receipt_' is: 721
invalid file count: 40
The highest number with prefix 'receipt_' is: 721
invalid file count: 40
The highest number with prefix 'receipt_' is: 723
invalid file count: 38
The highest number with prefix 'receipt_' is: 723
invalid file count: 39
The highest number with prefix 'receipt_' is: 723
invalid file count: 40
The highest number with prefix 'receipt_' is: 723
invalid file count: 37
The highest number with pref

KeyboardInterrupt: 